### Extract e mais algumas mudanças de transformação

In [76]:
# !pip install pandasql pyspark

In [164]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps
import numpy as np
from pyspark.sql import SparkSession
import plotly.express as px

algora = pd.read_csv('algoraquant.csv')
algora['vl_valor'] = algora['vl_valor'].fillna(0)
algora['vl_vpa'] = algora['vl_vpa'].fillna(0)

import pandas as pd


# Função para calcular o novo valor
def calcular_valor_normal(row):
    # Limpa e formata 'vl_vpa' antes de converter para float
    vpa_str = str(row['vl_vpa']).replace('.', '').replace(',', '.').strip()  # Remove milhar e converte decimal
    try:
        vpa = float(vpa_str)  # Converte para float
    except ValueError:
        return None  # Retorna None se a conversão falhar

    # Remove o '%' e substitui a vírgula por ponto, depois converte para float
    variacao = float(row['vl_valor'].replace('%', '').replace(',', '.'))

    # Calcula o novo valor
    novo_valor = vpa * (1 + variacao / 100)
    return novo_valor

# Filtra as linhas onde 'vl_valor' contém '%'
filtro_percentual = algora['vl_valor'].str.contains('%', na=False)

# Aplica o cálculo apenas para essas linhas
algora.loc[filtro_percentual, 'vl_valor'] = algora.loc[filtro_percentual].apply(calcular_valor_normal, axis=1)

import numpy as np

def clean_patrimonio(value):
    if pd.isnull(value) or value == 'NaN':
        return 0  # Mantém como NaN para tratar depois

    value = str(value).replace('.', '').replace(',', '.')  # Normaliza ponto e vírgula
    if '%' in value:  # Se houver porcentagem, retorna 0
        return 0

    value = value.replace('$', '')  # Remove o cifrão se existir

    try:
        return float(value)  # Converte para float
    except ValueError:
        return np.nan  # Se não puder ser convertido, retorna NaN


# Aplicando a função na coluna 'vl_patrimonio'
algora['vl_patrimonio'] = algora['vl_patrimonio'].apply(clean_patrimonio)

# Verificando o resultado após a limpeza

# Calculando a média excluindo valores iguais a 0
mean_value_excluding_zero = algora[algora['vl_patrimonio'] != 0]['vl_patrimonio'].mean()

# Substituindo os valores 0 pela média calculada
algora['vl_patrimonio'] = algora['vl_patrimonio'].replace(0, mean_value_excluding_zero)

import pandas as pd

# Supondo que você já tem o DataFrame 'algora'

# Lista de colunas para as quais queremos aplicar a conversão
colunas_para_convertir = [
    'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis', 'vl_valor',
    'vl_liquidez_media_diaria', 'vl_pvp', 'vl_dividendo', 'vl_yeld',
    'vl_soma_yield_3m', 'vl_soma_yield_6m', 'vl_soma_yield_12m',
    'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m',
    'vl_soma_yield_ano_corrente', 'vl_variacao_cotacao_mes', 'vl_rentabilidade_mes',
    'vl_rentabilidade', 'vl_patrimonio', 'vl_vpa', 'vl_pvpa', 'vl_vpa_yield',
    'vl_vpa_change', 'vl_vpa_rent_m', 'vl_vpa_rent', 'vl_ativos',
    'vl_volatility', 'vl_numero_cotista', 'vl_vacancia_fisica', 'vl_vacancia_financeira'
]

# Função para converter a coluna para float
def converter_para_float(coluna):
    try:
        # Remove os pontos (separadores de milhar)
        coluna = coluna.str.replace('.', '', regex=False)
        # Substitui a vírgula por ponto (separador decimal)
        coluna = coluna.str.replace(',', '.', regex=False)
        # Converte para float
        return coluna.astype(float)
    except Exception as e:
        print(f"Erro ao converter coluna: {e}")
        return coluna  # Retorna a coluna original em caso de erro

# Aplica a conversão para cada coluna na lista
for coluna in colunas_para_convertir:
    if coluna in algora.columns:
        algora[coluna] = converter_para_float(algora[coluna])



Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: could not convert string to float: '1.19 %'
Erro ao converter coluna: could not convert string to float: '3.46 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.15 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.27 %'
Erro ao converter coluna: could not convert string to float: '2.48 %'
Erro ao converter coluna: could not convert string to float: '12.91 %'
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao co

Pronto estamos prontos para começar as nossas análises exploratórias, aqui deixamos apenas valores decimais facieís de serem tratados,
ou seja ... que não nenhum tipo de porcentagem envolvido, ficando cerca de 10 colunas de valores a serem análisados de acordo com a estatística

In [165]:
algora = algora.drop(columns=['Unnamed: 0','valor_decimal', 'vl_vpa_rent','vl_vpa_rent_m','vl_vpa_change','vl_vpa_yield','vl_pvpa','vl_rentabilidade',
                              'vl_rentabilidade_mes','vl_variacao_cotacao_mes','vl_soma_yield_ano_corrente','vl_yeld', 'vl_soma_yield_3m',
                              'vl_soma_yield_6m', 'vl_soma_yield_12m','vl_ativos', 'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m','vl_vacancia_financeira','vl_vacancia_fisica'])
# prenchendo os outros títulos com os valores de Humanidade

categoria_renomeacao = {
    'Shoppings': 'Tijolo Comercial',
    #'Lajes Corporativas': 'Tijolo Comercial',
    'Imóveis Comerciais - Outros': 'Tijolo Comercial',
    #'Agências de Bancos': 'Tijolo Comercial',
    'Varejo': 'Tijolo Comercial',
    'Imóveis Industriais e Logísticos': 'Tijolo Logístico/Industrial',
    'Logística': 'Tijolo Logístico/Industrial',
    'Imóveis Residenciais': 'Tijolo Comercial',
    'Residencial': 'Tijolo Comercial',
    'Papéis': 'Papel',
    'Títulos e Val. Mob.': 'Papel',
    'Títulos e Valores Mobiliários': 'Papel',
    'Fundo de Fundos': 'Híbrido',
    'Misto': 'Híbrido',
    'Híbrido': 'Híbrido',
    'Fundo de Desenvolvimento': 'Desenvolvimento',
    'Incorporações': 'Desenvolvimento',
    'Hotéis': 'Tijolo Comercial',
    #'Hotel': 'Tijolo Comercial',
    'Serviços Financeiros Diversos': 'Setor Específico',
    'Educacional': 'Setor Específico',
    'Hospitalar': 'Setor Específico',
    'Hospital': 'Setor Específico',
    'Fiagro': 'Setor Específico',
    'Agricultura': 'Setor Específico',
    'Indefinido': 'Indefinido',
    'Indefinido2016': 'Indefinido',
    'Indefinido2015': 'Indefinido',
    'Indefinido2017': 'Indefinido',
    'Renda': 'Renda'
}

# Aplicando a renomeação no DataFrame 'algora'
algora['nm_setor'] = algora['nm_setor'].replace(categoria_renomeacao)


In [166]:
numeric_columns = ['vl_media_taxa_homicidios', 'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis']

# Aplicamos o 'groupby' apenas nas colunas numéricas
algora[numeric_columns] = algora.groupby('nm_titulo')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

algora['dt_inclusao'] = pd.to_datetime(algora['dt_inclusao'])

nm_titulo
0	CPTS11B
1	FIXX11
2	RBBV11
3	BZLI11
4	UBSR11
5	OUCY11
6	FOFT11
7	AFOF11
8	IFIE11
9	RBIV11
10	IFID11
11	QIFF11
12	EQIN11
13	ATSA11B
14	WPLZ11B
15	FIVN11
16	HMOC11
17	DMAC11
18	GRLV11
19	TRXL11
20	HGJH11
21	DOMC11
22	TBOF11
23	BBVJ11
24	BMLC11B
25	FFCI11
26	RDES11
27	FVBI11


Fundos que não existem ou pararam de existir ao longo dos anos ... 2016 até 202428	THRA11
29	GWIR11
30	AFCR11

### Distribuição de Dados Categóricas do Banco de Dados - FII - Correlação

In [168]:
import ipywidgets as widgets
import pandasql as ps
import pandas as pd


# Lista de categorias disponíveis
categorias = ['PAPEL', 'TIJOLO COMERCIAL', 'TIJOLO LOGíSTICO/INDUSTRIAL',
              'TIJOLO RESIDENCIAL',
              'HíBRIDO','DESENVOLVIMENTO', 'HOTéIS',
              'SETOR ESPECíFICO', 'INDEFINIDO', 'RENDA']

cat ='SETOR ESPECíFICO'

# Função que executa a query com base na seleção
def executar_query(categoria,dataframe):
    query = f"""
    SELECT *
    FROM algora
    WHERE
      dt_inclusao > '2021-08-15'
      AND UPPER(TRIM(nm_setor)) IN ('{categoria}')
    ORDER BY dt_inclusao DESC
    """

    # Executa a query usando pandasql
    gora = ps.sqldf(query, {"algora": dataframe})

    # Preenchendo valores nulos e ajustando a coluna 'vl_patrimonio'
    mean_value_excluding_zero = gora[gora['vl_patrimonio'] != 0]['vl_patrimonio'].mean()
    gora['vl_patrimonio'] = gora['vl_patrimonio'].fillna(0)
    gora['vl_patrimonio'] = gora['vl_patrimonio'].replace(0, mean_value_excluding_zero)

    # Exibe o número de linhas e colunas e descrição do DataFrame
    print(f"Temos {gora.shape[0]} linhas e {gora.shape[1]} colunas")

    # Obter a primeira e última data
    primeira_data = gora['dt_inclusao'].min()
    ultima_data = gora['dt_inclusao'].max()

    # Exibir os resultados
    print(f"A primeira data é: {primeira_data}")
    print(f"A última data é: {ultima_data}")

    return gora


# Executando a query inicial com o valor padrão
gora = executar_query(cat, algora)

gora_numeric = gora.drop(['nm_titulo','nm_setor','dt_inclusao'], axis = 1)

import plotly.express as px

# Gera a matriz de correlação
correlation_matrix = gora_numeric.corr()

# Criando o gráfico de correlação com limites ajustados de -1 a 1
fig = px.imshow(correlation_matrix,
                text_auto=True,
                zmin=-1, zmax=1,  # Define os limites de -1 a 1
                color_continuous_scale='RdBu')  # Usa uma escala de cores adequada

fig.show()

Temos 100 linhas e 14 colunas
A primeira data é: 2021-10-01 00:00:00.000000
A última data é: 2024-09-01 00:00:00.000000


De acordo com o conhecimento que obtive sobre as variáveis que temos
Precisaremos separar e realizar análises estatátiscas separadas para cada tipo de categoria .. então, vamos realizar uma nova categoria para um conjunto de subcategorias, então por exemplo:

['VAREJO',
    'IMóVEIS COMERCIAIS - OUTROS',
    'HOTéIS',
    'SERVIçOS FINANCEIROS DIVERSOS',
    'IMóVEIS RESIDENCIAIS']

  Serão os Comerciais, que indicam uma boa correlação entre as variáveis.

### Linha do Tempo Simplificada:

- **2016 - 2018**: Recuperação econômica lenta após recessão, juros elevados, FIIs em expansão inicial.
- **Junho de 2019**: Corte agressivo da Selic, cenário favorável para os FIIs.
- **2020 - 2021**: Pandemia de COVID-19 causa volatilidade, especialmente em FIIs de shoppings e escritórios.
- **2022 - 2024**: Retomada do mercado, elevação da Selic impacta o retorno dos FIIs, mas setores como logística e recebíveis ganham força.


### Modelo de Clusterização - Executar em Ambiente em Nuvem ou em PC Externo

In [84]:
from pyspark.sql import SparkSession

# Cria a sessão Spark com configurações ajustadas

spark = SparkSession.builder\
  .appName('recomendador FII pySpark')\
  .getOrCreate()


In [104]:
from pyspark.sql import functions as F

numeric_columns = [
    'vl_media_taxa_homicidios',
    'vl_avaliacao_media_ativos_fundo',
    'vl_quantidade_imoveis',
    'vl_valor',
    'vl_liquidez_media_diaria',
    'vl_pvp',
    'vl_dividendo',
    'vl_patrimonio',
    'vl_vpa',
    'vl_volatility',
    'vl_numero_cotista'
]

# Substituir os NaNs por 0 nas colunas numéricas
gora_spark = spark.createDataFrame(gora).na.fill(0, subset=numeric_columns)


aggregated_data = gora_spark.groupBy('nm_titulo').agg(
    F.mean('vl_media_taxa_homicidios').alias('mean_vl_media_taxa_homicidios'),
    F.mean('vl_avaliacao_media_ativos_fundo').alias('mean_vl_avaliacao_media_ativos_fundo'),
    F.mean('vl_quantidade_imoveis').alias('mean_vl_quantidade_imoveis'),
    F.mean('vl_valor').alias('mean_vl_valor'),
    F.mean('vl_liquidez_media_diaria').alias('mean_vl_liquidez_media_diaria'),
    F.mean('vl_pvp').alias('mean_vl_pvp'),
    F.mean('vl_dividendo').alias('mean_vl_dividendo'),
    F.mean('vl_patrimonio').alias('mean_vl_patrimonio'),
    F.mean('vl_vpa').alias('mean_vl_vpa'),
    F.mean('vl_volatility').alias('mean_vl_volatility'),
    F.mean('vl_numero_cotista').alias('mean_vl_numero_cotista')
)

In [105]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

def cria_pipeline_kmeans_cluster(aggregated_data):
    seed = 1224


    # Passo 2: Seleção das Features
    feature_columns = [
        'mean_vl_media_taxa_homicidios',
        'mean_vl_avaliacao_media_ativos_fundo',
        'mean_vl_quantidade_imoveis',
        'mean_vl_valor',
        'mean_vl_liquidez_media_diaria',
        'mean_vl_pvp',
        'mean_vl_dividendo',
        'mean_vl_patrimonio',
        'mean_vl_vpa',
        'mean_vl_volatility',
        'mean_vl_numero_cotista'
    ]

    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

    # Passo 3: Escalonamento
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

    # Passo 4: PCA (opcional, pode ser útil para visualizar em 2D, mas pode não ser necessário para clustering)
    pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')

    # Passo 5: KMeans
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster').setK(7).setSeed(seed)

    # Construção do Pipeline
    pipeline = Pipeline(stages=[assembler, scaler,pca, kmeans])

    # Treinamento do Pipeline
    pipeline_model = pipeline.fit(aggregated_data)

    # Aplicação do Pipeline
    predictions = pipeline_model.transform(aggregated_data)

    # Extração das Features PCA para Visualização
    from pyspark.ml.functions import vector_to_array
    pca_feature_xy = predictions.withColumn('x', vector_to_array('pca_features')[0])\
                                .withColumn('y', vector_to_array('pca_features')[1])\
                                .select(['x', 'y', 'cluster', 'nm_titulo'])

    # Visualização com Plotly
    import plotly.express as px
    fig = px.scatter(pca_feature_xy.toPandas(), x='x', y='y', color='cluster',
                     hover_data=['nm_titulo'])
    fig.show()

    return pipeline_model

In [106]:
pipeline_model = cria_pipeline_kmeans_cluster(aggregated_data)

### Iniciando recomendações de portifólio TOP 10 FII's para 6 meses futuros de acordo o perfil e variável de dividendo

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, StringIndexer, OneHotEncoder
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
import plotly.express as px



def cria_pipeline_kmeans_cluster(aggregated_data: DataFrame):
    seed = 1224

    # Passo 1: Indexação e Codificação de 'nm_setor'
    indexer = StringIndexer(inputCol='nm_setor', outputCol='nm_setor_indexed')
    encoder = OneHotEncoder(inputCols=['nm_setor_indexed'], outputCols=['nm_setor_encoded'])

    # Passo 2: Seleção das Features
    feature_columns = [
        'mean_vl_media_taxa_homicidios',
        'mean_vl_avaliacao_media_ativos_fundo',
        'mean_vl_quantidade_imoveis',
        'mean_vl_valor',
        'mean_vl_liquidez_media_diaria',
        'mean_vl_pvp',
        'mean_vl_dividendo',
        'mean_vl_patrimonio',
        'mean_vl_vpa',
        'mean_vl_volatility',
        'mean_vl_numero_cotista'
    ]

    # Adicionando a feature codificada ao conjunto de features
    assembler = VectorAssembler(inputCols=feature_columns + ['nm_setor_encoded'], outputCol='features')

    # Passo 3: Escalonamento
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features', withMean=True, withStd=True)

    # Passo 4: PCA (opcional)
    pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')

    # Passo 5: KMeans
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster').setK(7).setSeed(seed)

    # Construção do Pipeline
    pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, pca, kmeans])

    # Treinamento do Pipeline
    try:
        pipeline_model = pipeline.fit(aggregated_data)
    except Exception as e:
        print(f"Erro ao treinar o modelo: {e}")
        return None, None, None

    # Aplicação do Pipeline
    predictions = pipeline_model.transform(aggregated_data)

    # Mapeamento dos clusters para perfis
    predictions = predictions.withColumn(
        'perfil',
        F.when(F.col('cluster').isin([0, 1]), 'Arrojado')
         .when(F.col('cluster').isin([2, 3, 4]), 'Moderado')
         .otherwise('Conservador')
    )

    # Selecionar as top 10 por perfil
    windowSpec = Window.partitionBy('perfil').orderBy(F.desc('mean_vl_dividendo'))
    ranked_predictions = predictions.withColumn('rank', F.rank().over(windowSpec))
    top10_perfil = ranked_predictions.filter(F.col('rank') <= 10)

    # Definir futuras datas de recomendação
    future_dates = [
        '2024-10-01',
        '2024-11-01',
        '2024-12-01',
        '2025-01-01',
        '2025-02-01',
        '2025-03-01',
        '2025-04-01',
        '2025-05-01',
        '2025-06-01'
    ]
    future_recommendations = spark.createDataFrame(future_dates, StringType()).toDF('dt_recomendacao')

    # Selecionar apenas as colunas necessárias
    top10_recommendations = top10_perfil.select('perfil', 'nm_titulo')

    # Criar recomendações mensais
    recommendations = future_recommendations.crossJoin(top10_recommendations)

    # Exibir as recomendações
    recommendations.show(100, truncate=False)

    # Visualização opcional (apenas para um mês, por exemplo)
    # Filtrar por um mês específico
    sample_recommendation = recommendations.filter(F.col('dt_recomendacao') == '2024-10-01')

    # Visualizar com Plotly
    fig = px.scatter(
        sample_recommendation.toPandas(),
        x='perfil',
        y='nm_titulo',
        color='perfil',
        title='Recomendações TOP 10 FIIs para Outubro de 2024',
        labels={'perfil': 'Perfil de Investidor', 'nm_titulo': 'Nome do FII'}
    )
    fig.show()

    return pipeline_model, predictions, recommendations

# Chamar a função e capturar os retornos
pipeline_model, predictions, recommendations = cria_pipeline_kmeans_cluster(aggregated_data)

# Verificar se o modelo foi treinado corretamente
if pipeline_model:
    print("Modelo treinado com sucesso!")
else:
    print("Falha no treinamento do modelo.")

# Calcular o resumo dos clusters (opcional, se ainda não foi feito)
if predictions:
    cluster_summary = predictions.groupBy('cluster').agg(
        F.mean('mean_vl_media_taxa_homicidios').alias('avg_taxa_homicidios'),
        F.mean('mean_vl_avaliacao_media_ativos_fundo').alias('avg_avaliacao_ativos_fundo'),
        F.mean('mean_vl_quantidade_imoveis').alias('avg_quantidade_imoveis'),
        F.mean('mean_vl_valor').alias('avg_valor'),
        F.mean('mean_vl_liquidez_media_diaria').alias('avg_liquidez_media_diaria'),
        F.mean('mean_vl_pvp').alias('avg_pvp'),
        F.mean('mean_vl_dividendo').alias('avg_dividendo'),
        F.mean('mean_vl_patrimonio').alias('avg_patrimonio'),
        F.mean('mean_vl_vpa').alias('avg_vpa'),
        F.mean('mean_vl_volatility').alias('avg_volatility'),
        F.mean('mean_vl_numero_cotista').alias('avg_numero_cotista')
    ).orderBy('cluster')

    # Exibir o resumo dos clusters
    cluster_summary.show(truncate=False)


+---------------+-----------+---------+
|dt_recomendacao|perfil     |nm_titulo|
+---------------+-----------+---------+
|2024-10-01     |Arrojado   |BLMO11   |
|2024-10-01     |Arrojado   |BBFI11B  |
|2024-10-01     |Arrojado   |KEVE11   |
|2024-10-01     |Arrojado   |ALMI11   |
|2024-10-01     |Arrojado   |FAMB11B  |
|2024-10-01     |Arrojado   |DOMC11   |
|2024-10-01     |Arrojado   |RBOP11   |
|2024-10-01     |Arrojado   |HGPO11   |
|2024-10-01     |Arrojado   |EDFO11B  |
|2024-10-01     |Arrojado   |FPAB11   |
|2024-10-01     |Conservador|ERCR11   |
|2024-10-01     |Conservador|HUSI11   |
|2024-10-01     |Conservador|FIIB11   |
|2024-10-01     |Conservador|CXTL11   |
|2024-10-01     |Conservador|HCRI11   |
|2024-10-01     |Conservador|EURO11   |
|2024-10-01     |Conservador|NSLU11   |
|2024-10-01     |Conservador|FIIP11B  |
|2024-10-01     |Conservador|BRLA11   |
|2024-10-01     |Conservador|BTSG11   |
|2024-10-01     |Moderado   |GESE11B  |
|2024-10-01     |Moderado   |FAED11   |


Modelo treinado com sucesso!
+-------+-------------------+--------------------------+----------------------+------------------+-------------------------+-------------------+------------------+--------------------+--------------------+------------------+------------------+
|cluster|avg_taxa_homicidios|avg_avaliacao_ativos_fundo|avg_quantidade_imoveis|avg_valor         |avg_liquidez_media_diaria|avg_pvp            |avg_dividendo     |avg_patrimonio      |avg_vpa             |avg_volatility    |avg_numero_cotista|
+-------+-------------------+--------------------------+----------------------+------------------+-------------------------+-------------------+------------------+--------------------+--------------------+------------------+------------------+
|0      |1.8250694444444442 |0.13080357142857144       |2.1924603174603177    |74.68424603174604 |3219256.111904762        |0.6482539682539682 |0.5585714285714285|7.453898123184156E8 |1.3484011280066864E9|1.4286706349206348|33483.075396825

In [ ]:
# Exemplo: Visualizar recomendações para cada mês
for date in ['2024-10-01', '2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01',
             '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01']:
    month_recommendation = recommendations.filter(F.col('dt_recomendacao') == date)
    month_recommendation_pd = month_recommendation.toPandas()

    fig = px.scatter(
        month_recommendation_pd,
        x='perfil',
        y='nm_titulo',
        color='perfil',
        title=f'Recomendações TOP 10 FIIs para {date}',
        labels={'perfil': 'Perfil de Investidor', 'nm_titulo': 'Nome do FII'}
    )
    fig.show()


### Realizando Trade de FII em um modo conservador com base em netraulidade = 0, vender = -1 e comprar = 1

Quero realizar um estudo em cima de uma estratégia de fundo quantitativo, por exemplo você recebe 100.000 mil reais, e a recomendação através de regressão trabalhará em cima do retorno de dividendos observando as variáveis que temos dentro de nosso Banco de Dados, e você passará para a gente o quanto de tempo que manterá aplicado e no final a aplicação de acordo com os tipos de trades que fez, retornará a quantia que você terá e quanto ganhou em cima das aplicações que o "robo" fez

Também mostraremos os trades em FII que fizemos ao longo de todos os meses, por exemplo a carteira que escolhemos ter: Foi de 5 ativos mais ou menos, comprando ,vendendo e mantendo, dependendo do divendo e riscos que poderíamos ter na aplicação até finalizar este 1 ano realizando os trades.

In [107]:
import pandas as pd

# Ordenando os dados por ativo e data
gora['dt_inclusao'] = pd.to_datetime(gora['dt_inclusao'])
gora = gora.sort_values(by=['nm_titulo', 'dt_inclusao'])

# Verificar NaN e tratar
print(gora.isna().sum())
gora.fillna(method='ffill', inplace=True)  # Preenchendo com o valor anterior, por exemplo

nm_titulo                           0
vl_media_taxa_homicidios           84
vl_avaliacao_media_ativos_fundo    84
vl_quantidade_imoveis              84
nm_setor                            0
vl_valor                            6
vl_liquidez_media_diaria           28
vl_pvp                             33
vl_dividendo                        0
vl_patrimonio                       0
vl_vpa                             11
vl_volatility                      87
vl_numero_cotista                  84
dt_inclusao                         0
dtype: int64


<ipython-input-107-68c4f610567f>:9: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [108]:
# Criando lags para todas as variáveis de interesse
gora['vl_media_taxa_homicidios_lag1'] = gora.groupby('nm_titulo')['vl_media_taxa_homicidios'].shift(1)
gora['vl_avaliacao_media_ativos_fundo_lag1'] = gora.groupby('nm_titulo')['vl_avaliacao_media_ativos_fundo'].shift(1)
gora['vl_quantidade_imoveis_lag1'] = gora.groupby('nm_titulo')['vl_quantidade_imoveis'].shift(1)
gora['vl_valor_lag1'] = gora.groupby('nm_titulo')['vl_valor'].shift(1)
gora['vl_liquidez_media_diaria_lag1'] = gora.groupby('nm_titulo')['vl_liquidez_media_diaria'].shift(1)
gora['vl_pvp_lag1'] = gora.groupby('nm_titulo')['vl_pvp'].shift(1)
gora['vl_dividendo_lag1'] = gora.groupby('nm_titulo')['vl_dividendo'].shift(1)
gora['vl_patrimonio_lag1'] = gora.groupby('nm_titulo')['vl_patrimonio'].shift(1)
gora['vl_vpa_lag1'] = gora.groupby('nm_titulo')['vl_vpa'].shift(1)
gora['vl_volatility_lag1'] = gora.groupby('nm_titulo')['vl_volatility'].shift(1)
gora['vl_numero_cotista_lag1'] = gora.groupby('nm_titulo')['vl_numero_cotista'].shift(1)

# Remover as linhas com NaN gerados pelos lags
gora = gora.dropna()


In [109]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd

# Separando os dados em treino e teste
train = gora[gora['dt_inclusao'] < '2024-04-01']
test = gora[gora['dt_inclusao'] >= '2024-04-01']

# Features (variáveis explicativas) e Target (o que queremos prever)
X_train = train[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_train = train['vl_dividendo']

X_test = test[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_test = test['vl_dividendo']

# Criando e treinando o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

# Fazendo predições no conjunto de teste
y_pred = model.predict(X_test)

# Adicionando as predições ao DataFrame de teste para backtest
test['vl_dividendo_previsto'] = y_pred

# Calculando métricas de desempenho do modelo
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R²: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

# Exibindo as previsões e os valores reais para backtest
resultado = test[['dt_inclusao', 'nm_titulo', 'vl_dividendo', 'vl_dividendo_previsto']]
print("\nResultado do Backtest (Valores Reais x Previstos):")



R²: -3004.9501119286765
Mean Squared Error: 306469804.29403996
Mean Absolute Error: 5350.60133331613

Resultado do Backtest (Valores Reais x Previstos):


<ipython-input-109-6947194ce4e2>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [111]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Função para executar trade com otimizações
def executar_trade_optimizado(mes, saldo_atual, portifolio, ativos_para_comprar):
    # Vender ativos com previsão de queda nos dividendos
    ativos_para_vender = portifolio.loc[portifolio['vl_dividendo_previsto'] < portifolio['vl_dividendo']]

    # Calculando saldo após venda
    valor_venda = ativos_para_vender['valor_investido'].sum()
    saldo_atual += valor_venda

    # Atualizando portifólio removendo os ativos vendidos
    portifolio = portifolio.drop(ativos_para_vender.index)

    # Selecionar ativos para compra (evitando duplicação e garantindo diversificação)
    ativos_comprados = set(portifolio['nm_titulo'].tolist())
    novas_linhas = []

    # Evitar múltiplos loops e fazer tudo em um só bloco
    for ativo in ativos_para_comprar.itertuples():
        if ativo.nm_titulo not in ativos_comprados and saldo_atual > 0:
            investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
            novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto])
            saldo_atual -= investimento
            ativos_comprados.add(ativo.nm_titulo)

    # Verificar e garantir que há 6 ativos no portfólio
    while len(ativos_comprados) < 6 and saldo_atual > 0:
        for ativo in ativos_para_comprar.itertuples():
            if ativo.nm_titulo not in ativos_comprados:
                investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
                novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto])
                saldo_atual -= investimento
                ativos_comprados.add(ativo.nm_titulo)
            if len(ativos_comprados) >= 6:
                break

    # Adicionando novas linhas de uma só vez
    if novas_linhas:
        novas_linhas_df = pd.DataFrame(novas_linhas, columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto'])
        portifolio = pd.concat([portifolio, novas_linhas_df], ignore_index=True)

    return portifolio, saldo_atual

# Função otimizada para rodar o processo de investimento
def simular_investimento_optimizado(saldo_inicial, meses_investimento):
    saldo = saldo_inicial
    portifolio = pd.DataFrame(columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto'])

    # Pré-selecionar ativos conservadores uma vez para evitar múltiplas filtragens
    ativos_ordenados = test.sort_values(by='vl_volatility_lag1', ascending=True).head(20)

    for mes in pd.date_range('2024-10-01', periods=meses_investimento, freq='MS'):
        print(f"\nMês: {mes.strftime('%Y-%m')}")

        # Filtrar ativos disponíveis até a data do mês
        ativos_mes = ativos_ordenados[ativos_ordenados['dt_inclusao'] < mes]

        # Executar trade com portfólio otimizado
        portifolio, saldo = executar_trade_optimizado(mes, saldo, portifolio, ativos_mes)

        # Atualizando saldo com os dividendos recebidos
        dividendos_recebidos = (portifolio['vl_dividendo_previsto'] * portifolio['valor_investido']).sum()
        saldo += dividendos_recebidos

        print(f"Saldo Atual: R$ {saldo:.2f}")
        print(f"Portifólio Atual:\n{portifolio[['nm_titulo', 'valor_investido']]}")

    # Exibir saldo final após o período de investimento
    print(f"\nSaldo Final após {meses_investimento} meses: R$ {saldo:.2f}")
    return saldo

# Entrada de dados pelo usuário
saldo_inicial = float(input("Quanto você quer investir inicialmente (em reais)? "))
meses_investimento = int(input("Por quantos meses você deseja investir? "))

# Separar dados em treino e teste
train = gora[gora['dt_inclusao'] < '2024-04-01']
test = gora[gora['dt_inclusao'] >= '2024-04-01']

# Features (variáveis explicativas) e target (variável de saída)
X_train = train[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_train = train['vl_dividendo']

X_test = test[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_test = test['vl_dividendo']

# Criando e treinando o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Previsões
test['vl_dividendo_previsto'] = model.predict(X_test)

# Rodar simulação com otimizações
saldo_final = simular_investimento_optimizado(saldo_inicial, meses_investimento)


Quanto você quer investir inicialmente (em reais)? 30
Por quantos meses você deseja investir? 6

Mês: 2024-10
Saldo Atual: R$ 80403.54
Portifólio Atual:
  nm_titulo  valor_investido
0    NSLU11         1.500000
1    FAED11         1.425000
2   GESE11B         1.353750
3    HCRI11         1.286062
4    FCFL11         1.221759
5    NVHO11         1.160671
6    HUCG11         1.102638

Mês: 2024-11
Saldo Atual: R$ -150885.84
Portifólio Atual:
  nm_titulo  valor_investido
0    NSLU11         1.500000
1    FAED11         1.425000
2   GESE11B         1.353750
3    FCFL11         1.221759
4    HUCG11         1.102638
5    HCRI11      4020.299266
6    NVHO11      3819.284303

Mês: 2024-12
Saldo Atual: R$ -62567.69
Portifólio Atual:
  nm_titulo  valor_investido
0    NSLU11         1.500000
1    FAED11         1.425000
2   GESE11B         1.353750
3    FCFL11         1.221759
4    HUCG11         1.102638

Mês: 2025-01
Saldo Atual: R$ 17910.88
Portifólio Atual:
  nm_titulo  valor_investido
0    N

<ipython-input-111-98f125d2e878>:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-111-98f125d2e878>:43: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [113]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Função para executar trade com otimizações e observações
def executar_trade_optimizado_com_observacoes(mes, saldo_atual, portifolio, ativos_para_comprar):
    observacoes = []

    # Vender ativos com previsão de queda nos dividendos
    ativos_para_vender = portifolio.loc[portifolio['vl_dividendo_previsto'] < portifolio['vl_dividendo']]

    # Calculando saldo após venda
    valor_venda = ativos_para_vender['valor_investido'].sum()
    saldo_atual += valor_venda

    # Atualizando portifólio removendo os ativos vendidos e marcando trade = -1 (venda)
    portifolio.loc[ativos_para_vender.index, 'trade'] = -1
    observacoes.append(f"Vendidos {len(ativos_para_vender)} ativos devido à queda na previsão de dividendos.")
    portifolio = portifolio.drop(ativos_para_vender.index)

    # Selecionar ativos para compra (evitando duplicação e garantindo diversificação)
    ativos_comprados = set(portifolio['nm_titulo'].tolist())
    novas_linhas = []

    for ativo in ativos_para_comprar.itertuples():
        if ativo.nm_titulo not in ativos_comprados and saldo_atual > 0:
            investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
            novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto, 1])
            saldo_atual -= investimento
            ativos_comprados.add(ativo.nm_titulo)
            observacoes.append(f"Comprado ativo {ativo.nm_titulo} devido à boa previsão de dividendos e baixa volatilidade.")

    # Adicionar observação de manutenção
    if len(novas_linhas) == 0:
        observacoes.append("Nenhum novo ativo foi comprado, mantendo o portfólio atual.")

    # Verificar e garantir que há pelo menos 6 ativos no portfólio
    while len(ativos_comprados) < 6 and saldo_atual > 0:
        for ativo in ativos_para_comprar.itertuples():
            if ativo.nm_titulo not in ativos_comprados:
                investimento = min(saldo_atual / len(ativos_para_comprar), saldo_atual)
                novas_linhas.append([ativo.nm_titulo, investimento, ativo.vl_dividendo, ativo.vl_dividendo_previsto, 1])
                saldo_atual -= investimento
                ativos_comprados.add(ativo.nm_titulo)
                observacoes.append(f"Comprado ativo {ativo.nm_titulo} para garantir diversificação.")

            if len(ativos_comprados) >= 6:
                break

    # Adicionando novas linhas de uma só vez
    if novas_linhas:
        novas_linhas_df = pd.DataFrame(novas_linhas, columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto', 'trade'])
        portifolio = pd.concat([portifolio, novas_linhas_df], ignore_index=True)

    # Atualizando ativos mantidos (sem alterações) para trade = 0
    portifolio.loc[portifolio['trade'].isna(), 'trade'] = 0

    return portifolio, saldo_atual, observacoes

# Função otimizada para rodar o processo de investimento com observações
def simular_investimento_optimizado_com_observacoes(saldo_inicial, meses_investimento):
    saldo = saldo_inicial
    portifolio = pd.DataFrame(columns=['nm_titulo', 'valor_investido', 'vl_dividendo', 'vl_dividendo_previsto', 'trade'])

    # Pré-selecionar ativos conservadores uma vez
    ativos_ordenados = test.sort_values(by='vl_volatility_lag1', ascending=True).head(20)

    for mes in pd.date_range('2024-10-01', periods=meses_investimento, freq='MS'):
        print(f"\nMês: {mes.strftime('%Y-%m')}")

        # Filtrar ativos disponíveis até a data do mês
        ativos_mes = ativos_ordenados[ativos_ordenados['dt_inclusao'] < mes]

        # Executar trade e obter observações
        portifolio, saldo, observacoes = executar_trade_optimizado_com_observacoes(mes, saldo, portifolio, ativos_mes)

        # Atualizando saldo com os dividendos recebidos
        dividendos_recebidos = (portifolio['vl_dividendo_previsto'] * portifolio['valor_investido']).sum()
        saldo += dividendos_recebidos

        print(f"Saldo Atual: R$ {saldo:.2f}")
        print(f"Portifólio Atual:\n{portifolio[['nm_titulo', 'valor_investido', 'trade']]}")

        # Exibir observações do mês
        print("Observações do mês:")
        for observacao in observacoes:
            print(f"- {observacao}")

    # Exibir saldo final após o período de investimento
    print(f"\nSaldo Final após {meses_investimento} meses: R$ {saldo:.2f}")
    return saldo

# Entrada de dados pelo usuário
saldo_inicial = float(input("Quanto você quer investir inicialmente (em reais)? "))
meses_investimento = int(input("Por quantos meses você deseja investir? "))

# Separar dados em treino e teste
train = gora[gora['dt_inclusao'] < '2024-04-01']
test = gora[gora['dt_inclusao'] >= '2024-04-01']

# Features (variáveis explicativas) e target (variável de saída)
X_train = train[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_train = train['vl_dividendo']

X_test = test[['vl_avaliacao_media_ativos_fundo_lag1','vl_quantidade_imoveis_lag1','vl_valor_lag1','vl_liquidez_media_diaria_lag1','vl_dividendo_lag1','vl_patrimonio_lag1','vl_vpa_lag1','vl_volatility_lag1','vl_numero_cotista_lag1']]
y_test = test['vl_dividendo']

# Criando e treinando o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Previsões
test['vl_dividendo_previsto'] = model.predict(X_test)

# Rodar simulação com otimizações e observações
saldo_final = simular_investimento_optimizado_com_observacoes(saldo_inicial, meses_investimento)


Quanto você quer investir inicialmente (em reais)? 400
Por quantos meses você deseja investir? 5

Mês: 2024-10
Saldo Atual: R$ 1072047.18
Portifólio Atual:
  nm_titulo  valor_investido trade
0    NSLU11        20.000000     1
1    FAED11        19.000000     1
2   GESE11B        18.050000     1
3    HCRI11        17.147500     1
4    FCFL11        16.290125     1
5    NVHO11        15.475619     1
6    HUCG11        14.701838     1
Observações do mês:
- Vendidos 0 ativos devido à queda na previsão de dividendos.
- Comprado ativo NSLU11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo FAED11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo GESE11B devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo HCRI11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo FCFL11 devido à boa previsão de dividendos e baixa volatilidade.
- Comprado ativo NVHO11 devido à boa previsão de dividendos e baixa vol

<ipython-input-113-68e3f605f138>:113: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-113-68e3f605f138>:53: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

